# 推理引擎ONNX Runtime部署-预测单张图像

使用推理引擎 ONNX Runtime，读取 onnx 格式的模型文件，对单张图像文件进行预测。

同济子豪兄 https://space.bilibili.com/1900783

2022-8-22

## 应用场景

以下代码在需要部署的硬件上运行

只需把`onnx`模型文件发到部署硬件上，并安装 ONNX Runtime 环境，用几行代码就可以运行模型了。

## 导入工具包

In [1]:
import onnxruntime
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F
from torchvision import transforms

## 载入 onnx 模型，获取 ONNX Runtime 推理器

In [2]:
ort_session = onnxruntime.InferenceSession('resnet18_fruit30.onnx')

## 构造输入，获取输出结果

In [3]:
x = torch.randn(1, 3, 256, 256).numpy()

In [4]:
x.shape

(1, 3, 256, 256)

In [5]:
# onnx runtime 输入
ort_inputs = {'input': x}

# onnx runtime 输出
ort_output = ort_session.run(['output'], ort_inputs)[0]

注意，输入输出张量的名称需要和 torch.onnx.export 中设置的输入输出名对应

In [6]:
ort_output.shape

(1, 30)

In [8]:
# ort_output

## 预处理

In [9]:
# 测试集图像预处理-RCTN：缩放裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(256),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

### 载入测试图像

In [10]:
img_path = 'test_bananan.jpg'

In [11]:
# 用 pillow 载入
img_pil = Image.open(img_path)

In [14]:
# img_pil

## 运行预处理

In [15]:
input_img = test_transform(img_pil)

In [16]:
input_img.shape

torch.Size([3, 256, 256])

In [17]:
input_tensor = input_img.unsqueeze(0).numpy()

In [18]:
input_tensor.shape

(1, 3, 256, 256)

## ONNX Runtime预测

In [19]:
# ONNX Runtime 输入
ort_inputs = {'input': input_tensor}

In [20]:
# ONNX Runtime 输出
pred_logits = ort_session.run(['output'], ort_inputs)[0]
pred_logits = torch.tensor(pred_logits)

In [21]:
pred_logits.shape

torch.Size([1, 30])

In [22]:
pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

In [23]:
pred_softmax.shape

torch.Size([1, 30])

## 解析预测结果

In [24]:
# 取置信度最大的 n 个结果
n = 3

In [25]:
top_n = torch.topk(pred_softmax, n)

In [26]:
top_n

torch.return_types.topk(
values=tensor([[9.9972e-01, 1.1216e-04, 5.0886e-05]]),
indices=tensor([[28,  8, 29]]))

In [27]:
# 预测类别
pred_ids = top_n.indices.numpy()[0]

In [28]:
pred_ids

array([28,  8, 29])

In [29]:
# 预测置信度
confs = top_n.values.numpy()[0]

In [30]:
confs

array([9.9972135e-01, 1.1216481e-04, 5.0885566e-05], dtype=float32)

## 打印预测结果

In [31]:
# 载入类别和对应 ID
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()

In [33]:
# idx_to_labels

In [34]:
for i in range(n):
    class_name = idx_to_labels[pred_ids[i]] # 获取类别名称
    confidence = confs[i] * 100             # 获取置信度
    text = '{:<6} {:>.3f}'.format(class_name, confidence)
    print(text)

香蕉     99.972
椰子     0.011
黄瓜     0.005
